In [ ]:
import os
import csv
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from os.path import splitext

# Output directory for downloaded images
output_directory = r"D:\istock\civit\test"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Function to download images from URLs in CSV
def extract_main_image(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Extract main image URL from og:image meta tag
            main_image_url = soup.find('meta', property='og:image')['content']
            return main_image_url
        else:
            print(f"Failed to fetch content from {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return None

# Function to download images from URLs in CSV
def download_images_from_csv(csv_file):
    with open(csv_file, 'r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if 'ImageLink' in row:  # Check if the 'ImageLink' field exists in the row
                img_url = row['ImageLink']
                main_image_url = extract_main_image(img_url)
                if main_image_url:
                    try:
                        # Extract the image name from the URL
                        img_name = os.path.basename(urlparse(main_image_url).path)
                        img_path = os.path.join(output_directory, img_name)
                        # Download the main image
                        img_data = requests.get(main_image_url).content
                        with open(img_path, 'wb') as img_file:
                            img_file.write(img_data)
                        print(f"Image downloaded: {img_path}")
                    except Exception as e:
                        print(f"Error downloading image from URL {main_image_url}: {e}")
            else:
                print("Missing 'ImageLink' field in CSV row. Skipping...")

# Replace 'your_csv_file.csv' with the path to your CSV file
csv_file_path = r"C:\Users\karnika\Downloads\test.csv"
download_images_from_csv(csv_file_path)

print("All images downloaded successfully.")

In [ ]:
import os
import csv
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from os.path import splitext

def extract_title(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            title = None
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract title from response content
            title_element = soup.find('h1')
            if title_element:
                title = title_element.get_text(strip=True)
            
            return {'URL': url, 'Title': title if title else 'Not available'}
        else:
            print(f"Failed to fetch content from {url}. Status code: {response.status_code}")
            return {'URL': url, 'Title': 'Not available'}  # Assign 'Not available' if failed to fetch content
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return {'URL': url, 'Title': 'Not available'}  # Assign 'Not available' if an error occurs

def extract_titles_from_csv(input_csv, output_csv):
    with open(input_csv, 'r', newline='') as infile, open(output_csv, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = ['URL', 'Title']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for row in reader:
            if 'ImageLink' in row:  # Assuming the URL column is named 'URL' in the CSV file
                url = row['ImageLink']
                print(f"Extracting title from URL: {url}")
                extracted_data = extract_title(url)
                writer.writerow(extracted_data)
                print("Title extracted successfully.")
                print("=" * 50)

# Replace 'your_input_csv.csv' with the path to your input CSV file
input_csv_path = r"C:\Users\karnika\Downloads\test.csv"
# Replace 'your_output_csv.csv' with the desired path to your output CSV file
output_csv_path = r"C:\Users\karnika\Downloads\output.csv"

extract_titles_from_csv(input_csv_path, output_csv_path)